## Day 15: Dueling Generators

http://adventofcode.com/2017/day/15

### Part 1

Use generators to represent the generators.

In [1]:
import functools

def generator(divisor, factor, start):
    while True:
        start = (start * factor) % divisor
        yield start
        
dueling_generator = functools.partial(generator, 2147483647)

Have a look at what's being produced using the test input.

In [2]:
import itertools

test_generator_a = dueling_generator(16807, 65)
test_generator_b = dueling_generator(48271, 8921)

for a, b in itertools.islice(zip(test_generator_a, test_generator_b), 5):
    print(a, b)

1092455 430625591
1181022009 1233683848
245556042 1431495498
1744312007 137874439
1352636452 285222916


That seems to be working.

Now write a function to return the lowest 16 bits. Using bitwise operators rather than bin and slicing gives a decent speed up.

In [3]:
LOWEST_16_BITS_ON = 2**16 - 1

def lowest_16_bits(x):
    return x & LOWEST_16_BITS_ON

Restart the generators, otherwise they'll be five values in.

In [4]:
test_generator_a = dueling_generator(16807, 65)
test_generator_b = dueling_generator(48271, 8921)

for a, b in itertools.islice(zip(test_generator_a, test_generator_b), 5):
    print(bin(lowest_16_bits(a)), bin(lowest_16_bits(b)), lowest_16_bits(a) == lowest_16_bits(b))

0b1010101101100111 0b1101001100110111 False
0b1111011100111001 0b1000010110001000 False
0b1110001101001010 0b1110001101001010 True
0b1011011000111 0b1100110000000111 False
0b1001100000100100 0b10100000000100 False


That looks like it's doing what it should.

Total the number of times the lowest 16 bits match for forty million comparisons.

In [5]:
def forty_million_duels(gen_a, gen_b):
    return sum(1 for a, b in itertools.islice(zip(gen_a, gen_b), 40000000)
               if lowest_16_bits(a) == lowest_16_bits(b))

This is likely to take a while.

In [6]:
test_generator_a = dueling_generator(16807, 65)
test_generator_b = dueling_generator(48271, 8921)

%time forty_million_duels(test_generator_a, test_generator_b)

CPU times: user 26.3 s, sys: 7.69 ms, total: 26.3 s
Wall time: 26.3 s


588

Yes, it does. The answer is right, though.

<s>No speed up from pypy.</s> ~10x speed up from pypy when using bitwise operations. Parallelisation might help but for now get the answer for the problem input.

In [7]:
generator_a = dueling_generator(16807, 516)
generator_b = dueling_generator(48271, 190)

forty_million_duels(generator_a, generator_b)

597

### Part 2

This isn't too bad, rewrite the generator to filter for multiples of the given factor and generalise the number of duels.

In [8]:
def fussy_generator(divisor, multiplier, start, factor):
    while True:
        start = (start * multiplier) % divisor
        if start % factor == 0:
            yield start
        
fussy_dueling_generator = functools.partial(fussy_generator, 2147483647)

In [9]:
test_generator_a = fussy_dueling_generator(16807, 65, 4)
test_generator_b = fussy_dueling_generator(48271, 8921, 8)

for a, b in itertools.islice(zip(test_generator_a, test_generator_b), 5):
    print(a, b)

1352636452 1233683848
1992081072 862516352
530830436 1159784568
1980017072 1616057672
740335192 412269392


In [10]:
def duels(gen_a, gen_b, number_of_duels):
    return sum(1 for a, b in itertools.islice(zip(gen_a, gen_b), number_of_duels)
               if lowest_16_bits(a) == lowest_16_bits(b))

In [11]:
test_generator_a = fussy_dueling_generator(16807, 65, 4)
test_generator_b = fussy_dueling_generator(48271, 8921, 8)

%time duels(test_generator_a, test_generator_b, 5000000)

CPU times: user 14 s, sys: 3 µs, total: 14 s
Wall time: 14 s


309

Looking good. The drop in time taken is presumably because the number of cycles has been eighthed while the proportion of values generator A produces is roughly a quarter. 

In [13]:
generator_a = fussy_dueling_generator(16807, 516, 4)
generator_b = fussy_dueling_generator(48271, 190, 8)

%time duels(generator_a, generator_b, 5000000)

CPU times: user 13.5 s, sys: 0 ns, total: 13.5 s
Wall time: 13.5 s


303